# 要約 
このJupyter Notebookは、Kaggleのコンペティション「LMSYS - Chatbot Arena」において、与えられた埋め込みデータを用いてシンプルな分類器を訓練し、推論を行うことを目的としています。具体的には、異なるLLM（大規模言語モデル）による応答に対して、どちらのモデルの応答がユーザーに好まれるかを予測するモデルを構築します。

### 問題に取り組む背景
このNotebookは、Chatbot Arenaからの埋め込みを使用して、ユーザーがどのモデルの応答を選好するかを予測するというタスクに挑戦しています。モデルは、与えられた埋め込みを基に応答の優劣を判断する必要があります。

### 使用する手法とライブラリ
1. **埋め込みの計算**: 'Gemma 2'という大規模言語モデルを使用して、テキストデータから埋め込みを生成します。これをPyTorchとTransformersライブラリを用いて実装します。埋め込みは数値的な表現で、これが後の分類タスクに利用されます。

2. **分類器の訓練**: `CatBoostClassifier`を使用して、訓練データに基づいてモデルを構築します。CatBoostは、勾配ブースティングフレームワークの一種で、多くのデータセットにおいて良好な性能を示すとされています。パラメータ調整や学習率、イテレーション数の設定も行います。

3. **データの前処理とトークナイジング**: テストデータに対して、ユーザープロンプトとモデルの応答を適切に整形し、トークナイザーを使用してトークン化を行います。

4. **推論**: 訓練された分類器を用いて、テストデータに対する予測確率を計算します。

5. **出力形式**: 最終的な予測結果はCSV形式で保存され、提出可能な形式に整形されます。

全体として、このNotebookは深層学習と機械学習の技術を統合し、特に自然言語処理のタスクにおけるユーザーの好みを予測するための方法論を示しています。また、効率的なメモリ管理や多くの機能が備わったライブラリの利用を通じて、スムーズな実行を実現しています。

---


# 用語概説 
以下に、Jupyter Notebookの内容に関連し、初心者がつまずきやすい専門用語や概念に関する簡単な解説を示します。

### 専門用語の解説

1. **埋め込み (Embeddings)**:
   - 言語やデータを高次元のベクトルに変換する技術。語や文の意味を保持したまま数値表現することで、機械学習モデルに入力しやすくする。

2. **トークナイザー (Tokenizer)**:
   - テキストデータをモバイルや単語などのトークンに分割する役割を持つ。このプロセスは、自然言語処理タスクにおいて非常に重要で、モデルがテキストを理解できる形式に変換する。

3. **アテンションマスク (Attention Masks)**:
   - トランスフォーマーモデルで用いられるもので、どのトークンに注意を払うかを示すバイナリのマスク。例えば、パディングトークンに対しては注意を払わない設定など。

4. **量子化 (Quantization)**:
   - モデルのパラメータを省メモリかつ効率的に扱うために、浮動小数点から整数型に変換する技術。特に、GPUメモリや計算資源の制約がある環境でのモデルの効率化に役立つ。

5. **ガベージコレクション (Garbage Collection)**:
   - 使用しなくなったメモリを自動的に解放するプロセス。特にPythonでは、メモリ管理を向上させるために用いられる。

6. **自動混合精度 (Automatic Mixed Precision, AMP)**:
   - 訓練中に異なるデータ型（例えば、float32 と float16）を組み合わせて使用することで、計算速度を向上させつつメモリ使用量を削減する技術。

7. **早期停止 (Early Stopping)**:
   - モデルが訓練データに過剰適合するのを防ぐため、検証セットのスコアが改善しなくなった時点で訓練を中止する方法。これにより、汎化性能が向上する。

8. **スレッド (Thread)**:
   - プロセスの実行単位で、並行処理に用いる。特にデータ処理や計算を効率的に行うために、複数のスレッドを利用して並行にタスクを実行する。

9. **池 (Pool)**:
   - CatBoostにおいて、データを内部で効率的に処理するためのデータ構造。通常、データのサンプルや特徴量を含む。

10. **マルチクラス分類 (Multi-class Classification)**:
    - 複数のクラス（カテゴリ）から一つを選択する問題設定。例えば、複数のモデルの中から最も好まれるモデルを選ぶ場合など。

これらの用語は、具体的な実装やプロセスに関連しており、特に事前の実務経験がないと理解が難しい場合があるため、詳細な説明を加えました。

---


# Gemma 2 - 9b
ここでは、[こちら](https://www.kaggle.com/code/kishanvavdara/gemma-2-9b-part-1?scriptVersionId=186083288)から得られた計算された埋め込みを入力として使用して、シンプルな分類器を訓練し、テスト用の埋め込みを計算し、訓練した分類器を用いて推論を行います。それでは始めましょう！

もしこの内容が役に立ったと思ったら、いいねを押してください！

# ライブラリのインポート

In [ ]:
# bitsandbytesライブラリをインストールします。
# -qオプションは出力を抑制し、-Uオプションは最新バージョンにアップグレードします。
# --no-indexオプションはPyPIインデックスを使用しないことを指定します。
# --find-linksオプションは、指定したローカルのパスを使ってライブラリをインストールします。
!pip install -q -U bitsandbytes --no-index --find-links ../input/llm-detect-pip

# transformersライブラリをインストールします。
!pip install -q -U transformers --no-index --find-links ../input/libs-install

In [ ]:
# 必要なライブラリをインポートします。
import os  # オペレーティングシステムとの対話を行うためのライブラリ
import gc  # ガベージコレクタを扱うためのライブラリ
import re  # 正規表現を扱うためのライブラリ
from time import time  # 時間を計測するためのライブラリ

import torch  # PyTorchライブラリ
import transformers  # Hugging FaceのTransformersライブラリ
import sklearn  # Scikit-learnライブラリ
import random  # ランダム数生成のためのライブラリ
import numpy as np  # NumPyライブラリ（数値計算のため）
import pandas as pd  # Pandasライブラリ（データ操作のため）
import matplotlib.pyplot as plt  # データの可視化を行うためのライブラリ

from transformers import Gemma2ForCausalLM, GemmaTokenizer, BitsAndBytesConfig  # Gemma2モデルとトークナイザーのインポート

import time  # 再度時間計測用のライブラリをインポート
from catboost import CatBoostClassifier, Pool  # CatBoost分類器のインポート
from sklearn.model_selection import train_test_split  # データ分割のための関数をインポート
from sklearn.metrics import accuracy_score, log_loss  # 評価指標の関数をインポート

from torch.cuda.amp import autocast  # 自動混合精度訓練のためのモジュールをインポート
from threading import Thread  # スレッドを扱うためのライブラリ

# メモリ効率の良いシステムダイナミクスプログラミングを有効にします。
torch.backends.cuda.enable_mem_efficient_sdp(False)
torch.backends.cuda.enable_flash_sdp(False)

# CUDAが使用できない場合はエラーメッセージを表示します。
if (not torch.cuda.is_available()): 
    print("Sorry - GPU required!")  # GPUが必要です。申し訳ありません！

# 分類器の訓練

In [ ]:
# 訓練データをCSVファイルから読み込みます。
train_df = pd.read_csv('/kaggle/input/gemma-2-9b-part-1/train_embed.csv')

# 訓練用の埋め込みデータをNumPy配列として読み込みます。
train_embed = np.load('/kaggle/input/gemma-2-9b-part-1/gemma2_train_embed.npy')

# 'winner_model_a', 'winner_model_b', 'winner_tie'の列の中で最大の値のインデックスを取得し、
# それを'label'列としてデータフレームに追加します。
train_df.loc[:, 'label'] = np.argmax(train_df[['winner_model_a', 'winner_model_b', 'winner_tie']].values, axis=1)  # どのモデルが勝者かを示すラベルを作成します。

In [ ]:
# データを訓練セットとテストセットに分割します。
Targets = ['winner_model_a', 'winner_model_b', 'winner_tie']  # ターゲットとなる列を定義します。

y = train_df['label'].values  # ラベルの配列を取得します。
# train_test_split関数を使って、訓練データとテストデータのインデックスを分割します。
train_idx, test_idx = train_test_split(train_df.index, test_size=0.1, random_state=42, stratify=y)

# 訓練データとそのラベルを設定します。
X_train, y_train = train_embed[train_idx], train_df.iloc[train_idx]['label'].values
# テストデータとそのラベルを設定します。
X_test, y_test = train_embed[test_idx], train_df.iloc[test_idx]['label'].values

# 訓練データとテストデータの形状を表示します。
print(X_train.shape, y_train.shape)  # 訓練データの形状
print(X_test.shape, y_test.shape)  # テストデータの形状

In [ ]:
# ここでは、デフォルト設定で分類器を使用します。パラメータの調整を試みることもできます。

# CatBoostClassifierを初期化します。
model_cb = CatBoostClassifier(
    iterations=1000,  # 最大イテレーション数
    learning_rate=0.03,  # 学習率
    loss_function='MultiClass',  # マルチクラス分類の損失関数
    eval_metric='MultiClass',  # 評価指標
    early_stopping_rounds=10,  # 早期停止のためのラウンド数
    task_type='GPU',  # GPUを使用します。
    devices='0:1',  # 使用するGPUデバイスの指定
    verbose=100)  # 進捗状況の表示間隔

# モデルをフィッティングします。
model_cb.fit(X_train, y_train, 
              eval_set=(X_test, y_test),  # 検証セットとしてテストデータを指定します。
              early_stopping_rounds=50)  # 早期停止のためのラウンド数

In [ ]:
# テストデータに対する予測確率を取得します。
y_pred_proba = model_cb.predict_proba(X_test)
# テストデータに対する予測ラベルを取得します。
y_pred = model_cb.predict(X_test)

# モデルの評価を行います。
logloss = log_loss(y_test, y_pred_proba)  # ログ損失を計算します。
accuracy = accuracy_score(y_test, y_pred)  # 精度を計算します。
gc.collect()  # ガベージコレクションを実行してメモリを解放します。

# 結果を表示します。
print(f'Log Loss: {logloss:.3f}')  # ログ損失を表示します。
print(f'Accuracy: {accuracy:.3f}')  # 精度を表示します。

この分類器を推論に使用します。

# Gemma 2の読み込み

In [ ]:
# モデルのパスと設定を定義します。
MODEL_PATH = '/kaggle/input/gemma-2-9b-hf'  # Gemma 2のモデルパス
MAX_LENGTH = 1024  # 最大シーケンス長
BATCH_SIZE = 2  # バッチサイズ
    
# 使用するGPUデバイスを指定します。
device0 = torch.device('cuda:0')  # デバイス0を設定
device1 = torch.device('cuda:1')  # デバイス1を設定

# トークナイザーをモデルから読み込みます。
tokenizer = GemmaTokenizer.from_pretrained(MODEL_PATH)

# 4ビット量子化の設定を定義します。
bnb_config_4bit = BitsAndBytesConfig(
    load_in_4bit=True,  # 4ビットで読み込む設定
    bnb_4bit_compute_dtype=torch.float16,  # 計算のデータ型をfloat16に設定
    bnb_4bit_use_double_quant=False)  # ダブル量子化を使用しない設定

# モデル0を読み込みます。
model_0 = Gemma2ForCausalLM.from_pretrained(MODEL_PATH,
                                        revision="float16",  # float16バージョンを指定
                                        device_map='cuda:0',  # デバイスマップをデバイス0に設定
                                        quantization_config=bnb_config_4bit)  # 量子化設定を適用        

# モデル1を読み込みます。
model_1 = Gemma2ForCausalLM.from_pretrained(MODEL_PATH,
                                        revision="float16",  # float16バージョンを指定
                                        device_map='cuda:1',  # デバイスマップをデバイス1に設定
                                        quantization_config=bnb_config_4bit)  # 量子化設定を適用

In [ ]:
# 入力文字列を処理する関数を定義します。
def process(input_str):
    # 文字列の前後の角括弧を取り除きます。
    stripped_str = input_str.strip('[]')
    # 文字列をカンマで分割し、各文から前後の引用符を取り除きます。
    sentences = [s.strip('"') for s in stripped_str.split('","')]
    # 最後の文を返します。文がなければ空文字列を返します。
    return sentences[-1] if sentences else ''
  
# テストデータをCSVファイルから読み込みます。
test = pd.read_csv('/kaggle/input/lmsys-chatbot-arena/test.csv')

# 各カラムに対してprocess関数を適用し、結果を新しいカラムに保存します。
test.loc[:, 'prompt'] = test['prompt'].apply(process)  # ユーザープロンプトの処理
test.loc[:, 'response_a'] = test['response_a'].apply(process)  # モデルAの応答の処理
test.loc[:, 'response_b'] = test['response_b'].apply(process)  # モデルBの応答の処理

# テキストを指定の形式で組み合わせて新しいカラムを作成します。
test['text'] = '<start_of_turn>User prompt: ' + test['prompt'] +  '\n\nModel A :\n' + test['response_a'] +'\n\n----\n\nModel B:\n'  + test['response_b'] + '<end_of_turn><eos>'

# 生成されたテキストの最初の要素を表示します。
print(test['text'][0])  # 最初のテキストを表示します。

# トークナイズ

In [ ]:
# テストデータのテキストをトークナイズします。
tokens = tokenizer(test['text'].tolist(),
                   padding='max_length',  # 最大長さに合わせてパディングを行います。
                   max_length=MAX_LENGTH,  # 最大長さを設定します。
                   truncation=True,  # 長さが超えた場合は切り捨てます。
                   return_tensors='pt')  # PyTorchのテンソルを返します。

# トークナイズされたデータをデータフレームに変換します。
data = pd.DataFrame()
data['INPUT_IDS'] = [tensor.tolist() for tensor in tokens['input_ids']]  # 入力IDをリストとして保存
data['ATTENTION_MASKS'] = [tensor.tolist() for tensor in tokens['attention_mask']]  # アテンションマスクをリストとして保存

# データフレームの最初の2行を表示します。
data[:2]  # 最初の2行のデータを表示します。

# 埋め込みを取得する

In [ ]:
# 埋め込みを取得する関数を定義します。
def get_embeddings(df, model, device, batch_size=BATCH_SIZE):  
    # データフレームから入力IDとアテンションマスクを取得します。
    input_ids = torch.tensor(df['INPUT_IDS'].values.tolist(), dtype=torch.long)  # 入力IDをテンソルとして生成
    attention_mask = torch.tensor(df['ATTENTION_MASKS'].values.tolist(), dtype=torch.long)  # アテンションマスクをテンソルとして生成

    embed_list = []  # 埋め込みリストの初期化

    # バッチサイズに応じてデータを処理します。
    for start_idx in range(0, len(df), batch_size):
        end_idx = min(start_idx + batch_size, len(df))  # バッチの終了インデックスを計算
        batch_input_ids = input_ids[start_idx:end_idx].to(device)  # バッチの入力IDをデバイスに転送
        batch_attention_mask = attention_mask[start_idx:end_idx].to(device)  # バッチのアテンションマスクをデバイスに転送
        
        gc.collect()  # ガベージコレクションを実行してメモリを解放
        torch.cuda.empty_cache()  # CUDAメモリをクリアする

        with torch.no_grad():  # 勾配計算を無効にしてメモリを節約
            # モデルに入力して埋め込みを取得します。
            outputs = model(input_ids=batch_input_ids, attention_mask=batch_attention_mask, output_hidden_states=True)
            embed = outputs.hidden_states[-1]  # 最後の隠れ層の出力を取得
            embed_mean = torch.mean(embed, dim=1).cpu()  # 平均プーリングを行い、CPUに転送
            embed_list.append(embed_mean)  # 埋め込みリストに追加
            
            torch.cuda.empty_cache()  # CUDAメモリをクリアする
        
    # 埋め込みリストを結合して一つのテンソルを作成します。
    embeddings = torch.cat(embed_list, dim=0)
    return embeddings  # 埋め込みを返します。

# 埋め込みを計算する関数を定義します。
def compute_embed(df, model, device, results, index):
    results[index] = get_embeddings(df, model, device)  # 指定されたインデックスに埋め込みを格納します。

In [ ]:
# 処理開始時刻を記録します。
st = time.time()

N_SAMPLES = len(data)  # データのサンプル数を取得
half = round(N_SAMPLES / 2)  # サンプル数の半分を計算
sub1 = data.iloc[0:half].copy()  # データの最初の半分をコピー
sub2 = data.iloc[half:N_SAMPLES].copy()  # データの後半をコピー

results = {}  # 埋め込み結果を格納する辞書を初期化

# 2つのスレッドを作成し、それぞれのモデルとデバイスを指定します。
t0 = Thread(target=compute_embed, args=(sub1, model_0, device0, results, 0))  # モデル0のためのスレッド
t1 = Thread(target=compute_embed, args=(sub2, model_1, device1, results, 1))  # モデル1のためのスレッド

# スレッドを開始します。
t0.start()
t1.start()

# スレッドの終了を待ちます。
t0.join()
t1.join()

# 処理が完了したことを表示し、処理にかかった時間を表示します。
print(f"Processing complete. Total time: {time.time() - st:.2f} seconds")  # 処理の合計時間を表示します。

In [ ]:
# 2つのスレッドから得られた埋め込みを結合します。
test_embeddings = torch.cat([results[0], results[1]], dim=0)

# 最終的な埋め込みの形状を表示します。
test_embeddings.shape  # 埋め込みの形状を表示します。

In [ ]:
# ガベージコレクションを実行してメモリを解放します。
gc.collect()  

# モデル1を削除してメモリを解放します。
del model_1  
# モデル0を削除してメモリを解放します。
del model_0  

# CUDAメモリをクリアします。
torch.cuda.empty_cache()

# 推論

In [ ]:
# テスト埋め込みに対する予測確率を取得します。
preds = model_cb.predict_proba(test_embeddings.numpy())  # テスト埋め込みをNumPy配列に変換してモデルに渡します。
preds  # 予測確率を表示します。

# 提出用ファイルの作成

In [ ]:
# サンプル提出ファイルをCSVから読み込みます。
sample_sub = pd.read_csv('/kaggle/input/lmsys-chatbot-arena/sample_submission.csv')

# ターゲット列に予測結果を代入します。
sample_sub[Targets] = preds

# 最終的な提出用データを表示します。
display(sample_sub)  # 提出用データの表示。

In [ ]:
# 提出用ファイルをCSV形式で保存します。
sample_sub.to_csv('submission.csv', index=False)  # インデックスなしで'submission.csv'として保存します。

# まとめ 

これで終わりです！アイデアを共有したかっただけです！分類器の調整や他の分類器の使用を試してみてください。ありがとうございました！

もし何かを学んだなら、ぜひいいねを押してください:)

---

# コメント 

> ## superferg
> 
> Gemma2を直接分類用に訓練する予定はありますか？私はローカルのバリデーションセットで良いスコアを得たのですが、公開リーダーボードで対応するスコアを達成できていません。Llama3の推論コードが直接Gemma2に変更されてしまっているのではないかと疑っています。
> 
> [詳細](https://www.kaggle.com/competitions/lmsys-chatbot-arena/discussion/518408)
> 
> 
> > ## Valentin Werner
> > 
> > 推論ノートブックで同じ処理をしていることを確認してください。トークナイザー、モデルクラス、入力処理はすべて同一であるべきです。私の場合、提出ノートブック内でCVをテストして、同様のスコアが得られているかを見るのが役に立ちます。提出時に異なる量子化を行うと、トレーニング時とは異なる結果が出ることがあるかもしれませんが（ただし0.95から1.0のような乖離ではないにしても）。
> > 
> > 

---